In [ ]:
import random
import sympy

x = 32; n = 6; l = 3; m = 2;

while True:
  q = sympy.randprime(2**x, 2**(x+1))
  if sympy.isprime(2**x*q+1):
    p = 2**x*q+1
    break

p,q

R=Integers(p-1)

Rp = Integers(p)

l1 = 2; l2 = 1;

#generate all the polynomials for the secret and public key (code from Guo)
beta = [[[R.random_element() for i in range(n+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

f = [R.random_element() for i in range(l+1)]
h = [R.random_element() for i in range(l+1)]

phi = [[convolution(f, beta[j1][j2]) for j2 in range(l2+1)] for j1 in range(l1+1)]

psi = [[convolution(h, beta[j1][j2]) for j2 in range(l2+1)] for j1 in range(l1+1)]

E_phi = [R.random_element() if i != 0 and i != n+l else R.zero() for i in range(n+l+1)]
E_psi = [R.random_element() if i != 0 and i != n+l else R.zero() for i in range(n+l+1)]

#I just checked manually that R_0 and R_n were even (it doesn't look like Guo actually included this requirement)
R_0 = R.random_element()
R_n = R.random_element()
R_0, R_n

N_0 = [[[R.zero() if i != 0 else R_0 * beta[j1][j2][0] for i in range(n+l+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

N_n = [[[R.zero() if i != n+l else R_n * beta[j1][j2][n] for i in range(n+l+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

Phi = [[[R.zero() if i == 0 or i == n+l else phi[j1][j2][i] for i in range(n+l+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

Psi = [[[R.zero() if i == 0 or i == n+l else psi[j1][j2][i] for i in range(n+l+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

P = [[[R_0 * Phi[j1][j2][i] if (j1 != 0 or j2 != 0)else R_0 * (Phi[j1][j2][i] - E_phi[i]) for i in range(n+l+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

Q = [[[R_n * Psi[j1][j2][i] if (j1 != 0 or j2 != 0) else R_n * (Psi[j1][j2][i] - E_psi[i]) for i in range(n+l+1)] for j2 in range(l2+1)] for j1 in range(l1+1)]

#generate random message to be signed
mu = R.random_element(); mu

S = PolynomialRing(R, "x_0")

#generate required components to produce an honest signature
a = R_0 * S(f)(mu); a

b = R_n * S(h)(mu); b

c = R_n * (S(h)(mu)* f[0] - S(f)(mu)*h[0]); c

d = R_0 * (S(h)(mu)*f[l]- S(f)(mu)*h[l]); d

e = R_0 * R_n * (S(h)(mu)* S(E_phi)(mu) - S(f)(mu) * S(E_psi)(mu)); e

#verifies that everything was defined properly and the signature will pass verification
[[a*S(Q[j1][j2])(mu) - b*S(P[j1][j2])(mu) - c*S(N_0[j1][j2])(mu) - d*S(N_n[j1][j2])(mu) for j2 in range(l2+1)] for j1 in range(l1+1)]

honestbase = Rp.random_element(); honestbase

#produce and print the honest signature
honestsig = [honestbase**a, honestbase**b, honestbase**c, honestbase**d, honestbase**e]; honestsig

#find a generator
while True:
  gen = Rp.random_element()
  if ((gen != 0) and (gen**((p-1)/2)!=1) and (gen**((p-1)/q)!= 1)):
    break

#produce element of order 8 using the generator
forgebase = gen**((p-1)/8)
gen, forgebase

#system of equations to be solved
Mx = Matrix(R, (l1+1)*(l2+1), 4, [
    [S(Q[j1][j2])(mu),
     -S(P[j1][j2])(mu),
     -S(N_0[j1][j2])(mu),
     -S(N_n[j1][j2])(mu)] for j1 in range(l1+1) for j2 in range(l2+1)
])
Mx

R8 = Integers(8)

#reduce the system of equations mod 8
M8 = (Mx % 8).change_ring(R8); M8

#solve the system mod 8 (Guo's code modified from 2^x to 2^3)
for i in range(3):
    v8 = vector([R8.one()*2^i] + [R8.zero() for _ in range((l1+1)*(l2+1)-1)])
    try:
        ans8 = M8 \ v8
    except ValueError:
        continue
    else:
        break
ans8, v8

#produce and print the forged signature
forgesig = [forgebase**ans8[0], forgebase**ans8[1], forgebase**ans8[2], forgebase**ans8[3], forgebase**v8[0]]; forgesig

